### Emergency Modeling

In [2]:
import pandas as pd
import os
from datetime import timedelta
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
from sklearn.preprocessing import LabelEncoder
from base.influx_utils import fetch_all_sensor_data
from occupancy_model import (
    prepare_data_for_occupancy_model,
    train_occupancy_model,
    map_sensor_to_room,
    calculate_times_in_each_room
)
from base.minio_utils import save_model_to_minio

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [4]:
# Fetch sensor data
sensor_data = fetch_all_sensor_data(start_hours=24 * 7 * 5, interval_hours=24 * 7)
sensor_data_df = prepare_data_for_occupancy_model(sensor_data)

In [5]:
sensor_data_df.head(10)

,sensor,bucket,timestamp,value,type,sensor_encoded
2249,livingroom_magnetic_switch,1_4_11,2024-11-28 12:44:23.000,livingroomdoor,sensor,3
0,kitchen_PIR,1_5_9,2024-11-28 12:48:28.413,kitchen,sensor,1
1,kitchen_PIR,1_5_9,2024-11-28 12:50:00.695,kitchen,sensor,1
2,kitchen_PIR,1_5_9,2024-11-28 12:50:09.487,kitchen,sensor,1
3,kitchen_PIR,1_5_9,2024-11-28 12:56:04.021,kitchen,sensor,1
4,kitchen_PIR,1_5_9,2024-11-28 12:56:26.944,kitchen,sensor,1
5,kitchen_PIR,1_5_9,2024-11-28 12:56:41.850,kitchen,sensor,1
6,kitchen_PIR,1_5_9,2024-11-28 12:56:54.202,kitchen,sensor,1
2250,livingroom_magnetic_switch,1_4_11,2024-11-28 12:57:06.000,livingroomdoor,sensor,3
7,kitchen_PIR,1_5_9,2024-11-28 12:57:08.824,kitchen,sensor,1


In [6]:
sensor_data_df_aggregated = calculate_times_in_each_room(sensor_data_df) # It will also exclude door events after aggregation
sensor_data_df_aggregated.head(10)

,group_id,room,start_time,end_time,duration,duration_seconds
1,2,kitchen,2024-11-28 12:48:28.413,2024-11-28 12:56:54.202,0 days 00:08:25.789000,505.789
3,4,kitchen,2024-11-28 12:57:08.824,2024-11-28 21:36:06.797,0 days 08:38:57.973000,31137.973
5,6,kitchen,2024-11-29 16:03:53.625,2024-11-29 16:06:44.833,0 days 00:02:51.208000,171.208
7,8,kitchen,2024-11-29 16:07:01.214,2024-11-29 16:11:49.475,0 days 00:04:48.261000,288.261
9,10,kitchen,2024-11-29 16:11:56.395,2024-11-29 16:26:04.475,0 days 00:14:08.080000,848.080
11,12,kitchen,2024-11-29 16:28:03.347,2024-11-29 16:30:01.818,0 days 00:01:58.471000,118.471
13,14,kitchen,2024-11-29 16:30:12.217,2024-11-29 16:30:35.666,0 days 00:00:23.449000,23.449
15,16,kitchen,2024-11-29 16:31:07.171,2024-11-29 16:31:07.171,0 days 00:00:00,0.000
17,18,kitchen,2024-11-29 16:34:58.560,2024-11-29 16:40:26.580,0 days 00:05:28.020000,328.020
19,20,kitchen,2024-11-29 16:50:58.135,2024-11-29 17:03:18.713,0 days 00:12:20.578000,740.578


In [7]:
sensor_data_df_aggregated.groupby("room").count()

,group_id,start_time,end_time,duration,duration_seconds
room,,,,,
bathroom,38,38,38,38,38
kitchen,67,67,67,67,67
livingroom,59,59,59,59,59


In [8]:
stats_df = train_occupancy_model(sensor_data_df)
save_model_to_minio(stats_df, "occupancy")